In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.backends.mps.is_built()

True

In [4]:
torch.backends.mps.is_available()

True

In [5]:
device = torch.device("mps")

In [6]:
import pandas as pd
import os
from torchvision.io import read_image
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
from torch.utils.data import Dataset
import math

labels = pd.read_csv("labels.csv", index_col=0)

class PlayerDataset(Dataset):
    def __init__(self, labels=labels, img_dir="NBA Players"):
        self.img_labels = labels
        self.img_dir = img_dir
        self.img_files = [os.path.join(path, name) for path, subdirs, files in os.walk("NBA Players") for name in files if name.endswith("jpg")]
        
        self.filter_teams(["OKC", "WAS", "LAL"])
        
        self.img_labels["team_number"] = self.img_labels["team"].astype("category").cat.codes
        self.augments = [RandomHorizontalFlip(1), RandomRotation(90), RandomVerticalFlip(1)]

    def __len__(self):
        return len(self.img_files) * (len(self.augments) + 1)
    
    def filter_teams(self, teams):
        filtered = self.img_labels[self.img_labels["team"].isin(teams)]
        filtered_names = list(filtered["folder_name"])
        new_img_files = []
        for img_path in self.img_files:
            player_name = img_path.split("/")[1]
            if player_name in filtered_names:
                new_img_files.append(img_path)
        self.img_files = new_img_files
        self.img_labels = filtered.copy()
        
    
    def teams(self):
        return self.img_labels["team"].unique()

    def __getitem__(self, idx):
        augment = math.floor(idx / len(self.img_files))
        idx = idx % len(self.img_files)
            
        img_path = self.img_files[idx]
        image = read_image(img_path)
        
        player_name = img_path.split("/")[1]
        label = self.img_labels[self.img_labels["folder_name"] == player_name]["team_number"].iloc[0]
        
        if augment > 0:
            image = self.augments[augment - 1].forward(image)

        return image, int(label)

In [7]:
data = PlayerDataset()

In [8]:
teams = data.teams()

In [9]:
len(teams)

7

In [10]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])

In [11]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128
EPOCHS = 10

train = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, classes):
        super(NeuralNetwork, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 8, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(8, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(.1),
        )
        
        self.dense = nn.Sequential(
            nn.Linear(32 * 28 * 28, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, len(classes))
        )

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, 1) 
        x = self.dense(x)
        return x

In [13]:
model = NeuralNetwork(teams).to(device)

In [14]:
model

NeuralNetwork(
  (cnn): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.1, inplace=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=7, bias=True)
  )
)

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [ ]:
size = len(train.dataset)

for epoch in range(EPOCHS):
    for batch, (images, labels) in enumerate(train): 
        optimizer.zero_grad()
        
        # Compute prediction and loss
        images = images.to(device)
        pred = model(images.float())
        
        labels = labels.to(device)
        loss = loss_fn(pred, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

    loss, current = loss.item(), batch * len(images)
    print(f"loss: {loss:>7f}  [{epoch / EPOCHS:>3f}%]")

loss: 1.898536  [0.000000%]
loss: 1.958914  [0.100000%]
loss: 1.929461  [0.200000%]
loss: 1.873349  [0.300000%]
loss: 1.819665  [0.400000%]
loss: 1.800894  [0.500000%]
loss: 1.810111  [0.600000%]
loss: 1.688251  [0.700000%]
loss: 1.356767  [0.800000%]


In [ ]:
all_preds = list()
all_labels = list()

with torch.no_grad():
    for batch, (images, labels) in enumerate(test):
        
        images = images.to(device)
        outputs = model(images.float())
        
        _, preds = torch.max(outputs.data, 1)
        
        all_labels.append(labels)
        all_preds.append(preds)

In [ ]:
import numpy as np

preds = np.concatenate([p.cpu().numpy() for p in all_preds])
labels = np.concatenate([p.cpu().numpy() for p in all_labels])

In [ ]:
((preds == labels).sum()) / len(labels)

In [ ]:
labels

In [ ]:
train.dataset[2][0].shape

In [ ]:
[t[0].shape for t in list(train.dataset) if t[0].shape[1] > 256]

In [ ]:
[os.path.join(path, name) for path, subdirs, files in os.walk("NBA Players") for name in files if name.endswith("jpg")][14]